# DAQ-card data analysis

### This script analyzes the recorded serial output from the daq card.


In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from bisect import bisect_left
import itertools
import time
import datetime

In [2]:
# variables for configuration

data_file = "20171110-1240_Thresh_scan_start-5_stop-200_step-5_sec_per_rec_120_detector_mode_2E.txt"

max_coincidence_time_s = 100*10**(-9)  # 100 ns

active_channels = [1,2,3]


In [3]:
# Functions
def simplecount(filename):
    lines = 0
    for line in open(filename, 'U'):
        lines += 1
    return lines

def check_line_for_corruption(line):
    # check for obvious corruption
    if len(line.split(' ')) is not 16:
        return True
    # split line
    split_line = line.split(" ")
    # check each field for currupted-ness
    for i in range(len(split_line)):
        length = len(split_line[i])
        if i in (0,9):
            if length != 8:
                return True
        if i in range(1,9):
            if length != 2:
                return True
        if i == 10:
            if length != 10:
                return True
        if i == 11:
            if length != 6:
                return True
        if i == 12:
            if length != 1:
                return True
        if i == 13:
            if length != 2:
                return True
        if i == 14:
            if length != 1:
                return True
        if i == 15:
            if length != 6:
                return True
        
    # all checks passed
    return False

def findClosest(list_to_search, value):
    # bisect only works because we know that our list is sorted
    pos = bisect_left(list_to_search, value)
    if pos == 0:
        return list_to_search[0]
    if pos == len(list_to_search):
        return list_to_search[-1]
    before = list_to_search[pos - 1]
    after = list_to_search[pos]
    if after - value < value - before:
       return after
    else:
       return before

def parse_edge_data_forgetfull_parser(file_lines):
    edge_and_tirgger_pos = [[] for i in range(9)]

    previous_PPS = -1000
    current_PPS = -1000

    rollover_counter_trig = 0
    rollover_counter_pps = 0
    old_unadjusted_time_trig = 0
    old_unadjusted_time_pps = 0

    # list for saving our data
    edge_and_tirgger_pos = [[] for i in range(9)]

    lines_to_read = len(file_lines)
    print("Lines to read: "+ str(lines_to_read))

    print("Starting to parse section")
    line_count = 0
    discarded_lines = 0
    line_buffer = []
    event_lengths = []
    discarded_event_lengths = []

    for into_buffer_line in file_lines:
        line_count += 1
        if (line_count % (lines_to_read//20)) == 0:
            print("Percentage of section parsed: " + str(round( float(line_count)*100 / float(lines_to_read), 2)))
        # buffer lines to make sure the event is not corrupted
        # empty the buffer when a corrupt line was found
        if check_line_for_corruption(into_buffer_line):
            discarded_lines += 1 + len(line_buffer)
            discarded_event_lengths.append(len(line_buffer)+1)
            line_buffer = []
            continue
        # if the buffer was empty add the line and continue
        if len(line_buffer) == 0:
            line_buffer.append(into_buffer_line)
            continue
        # if the event has come to an end; parse it, otherwise just buffer the line
        new_timestamp = int(into_buffer_line.split(" ")[0], 16)
        old_timestamp = int(line_buffer[0].split(" ")[0], 16)
        if new_timestamp == old_timestamp:
            line_buffer.append(into_buffer_line)    
        else:
            event_lengths.append(len(line_buffer))
            for current_line in line_buffer:
                # split line
                split_line = current_line.split(" ")        

                # parse the line
                try:
                    # get the current pps and trigger
                    pps = int(split_line[9], 16)
                    trigger_clock = int(split_line[0], 16)
                    # check for rollovers
                    if old_unadjusted_time_pps > pps:
                        rollover_counter_pps += 1
                    if old_unadjusted_time_trig > trigger_clock:
                        rollover_counter_trig += 1
                    # adjust for rollovers
                    old_unadjusted_time_pps = pps
                    pps = rollover_counter_pps * 2**36 + pps
                    old_unadjusted_time_trig = trigger_clock
                    trigger_clock = rollover_counter_trig * 2**36 + trigger_clock

                    # save pps
                    if pps != current_PPS:
                        previous_PPS = current_PPS
                        current_PPS = pps
                    # check if we already have a previous pps
                    if previous_PPS < 0:
                        continue

                    # calculate the UTC time
                    day = int(split_line[11][0:2])
                    month = int(split_line[11][2:4])
                    year = int(split_line[11][4:6])
                    hour = int(split_line[10][0:2])
                    minute = int(split_line[10][2:4])
                    second = float(split_line[10][4:10])
                    second = int(round(second + float(split_line[15])/1000 ))
                    time_from_gps = datetime.datetime(year+2000,
                                                    month,
                                                    day,
                                                    hour,
                                                    minute,
                                                    second,
                                                    tzinfo=None)
                    unix_seconds = (time_from_gps - datetime.datetime(1970,1,1,tzinfo=None)).total_seconds()
                    clock_rate = current_PPS - previous_PPS
                    sub_sec = float(trigger_clock - current_PPS) / float(clock_rate)    

                    # convert stuff into the binary representation
                    bin_rep = []
                    for i in range(1,9):
                        bin_rep.append(bin(int(split_line[i],16))[2:].zfill(8))

                    # check if the daq triggered
                    triggered = False
                    if bin_rep[0][0] == '1':
                        triggered =  True
                    if triggered:
                        edge_and_tirgger_pos[0].append(unix_seconds+sub_sec)

                    # check rising and falling edges
                    for i in range(len(bin_rep)):
                        bin_number = bin_rep[i]
                        if bin_number[2] == '1':
                            edge_nano_secs = float(int(bin_number[3:],2))/32 * 24
                            edge_sub_secs = edge_nano_secs * 10**(-9)
                            edge_and_tirgger_pos[i+1].append(unix_seconds+sub_sec+edge_sub_secs)
                except ValueError:
                    print("Caught a value error, that had slipped through the corruption check. Line: "+ str(line_count))
                    print("Errorous line: "+ current_line)
                    discarded_lines += 1
                    break
            # empty the buffer and pre-save the next line
            line_buffer = []
            line_buffer.append(into_buffer_line)  

    print("Percentage of lines that were corrputed: "+str(100.0*discarded_lines/float(line_count)))
    print("Average event length: {:.2f}".format(np.mean(event_lengths)))
    print("Median event length: {:.0f}".format(np.median(event_lengths)))
    print("Stdabw. event length: {:.2f}".format(np.std(event_lengths)))
    print("Average discarded event length: {:.2f}".format(np.mean(discarded_event_lengths)))
    print("Median discarded event length: {:.0f}".format(np.median(discarded_event_lengths)))
    print("Stdabw. discarded event length: {:.2f}".format(np.std(discarded_event_lengths)))
    return edge_and_tirgger_pos

def calculate_correlations_from_edge_data(edge_data, max_coincidence_time_s, active_channels=[1,2,3]):
    # calculate correlations

    combinations = list(itertools.combinations(detector_area.keys(), 2))
    correlations_to_do = []
    for i in range(len(detector_area.keys())):
        correlations_to_do.append((combinations[i], detector_area.keys()[2-i]))

    correlations = {}
    for corr_tupel in correlations_to_do:
        (key1, key2), key3 = corr_tupel
        # do the first correlation
        base_correlations = {'base_time':[], 'diff':[]}

        for key1_time in edge_data[key1*2+1]:
            closest_key2_time = findClosest(edge_data[key2*2+1], key1_time)
            diff = closest_key2_time - key1_time
            if (abs(diff) <= max_coincidence_time_s):
                base_correlations['base_time'].append(key1_time)
                base_correlations['diff'].append(diff)
        correlations["CH"+str(key1)+"<->CH"+str(key2)] = base_correlations

        second_level_correlation = {'base_time':[], 'diff':[]}
        # do second correlation
        for base_time in base_correlations['base_time']:
            closest_key3_time = findClosest(edge_data[key3*2+1], base_time)
            diff = closest_key3_time - base_time
            if (abs(diff) <= max_coincidence_time_s):
                second_level_correlation['base_time'].append(base_time)
                second_level_correlation['diff'].append(diff)

        correlations["[CH"+str(key1)+"<->CH"+str(key2)+"]<->CH"+str(key3)] = second_level_correlation
    
    return correlations

In [4]:
print("Counting lines")
lines_to_read = simplecount(data_file)
print("Lines to read: "+ str(lines_to_read))

print("Starting to parse the file")
line_count = 0
recording_secs_per_setting = 0
current_threshold = 0

threshold_list = []
count_lists = []
edge_data_list = []
edge_data_lines = []

for current_line in open(data_file, 'U'):
    line_count += 1
    if (line_count % (lines_to_read//20)) == 0:
        print("Percentage of file read: " + str(round( float(line_count)*100 / float(lines_to_read), 2)))
    
    if "recording_secs_per_setting:" in current_line:
        split_line = current_line.replace("\n","").split(":")
        recording_secs_per_setting = float(split_line[len(split_line)-1])
        continue
    
    if "Current Threshold:" in current_line:
        split_line = current_line.replace("\n","").split(":")
        current_threshold = int(split_line[len(split_line)-1])
        if len(edge_data_lines) != 0:
            edge_data_list.append(parse_edge_data_forgetfull_parser(edge_data_lines))
            edge_data_lines = []
        continue
    
    if "DS S0" in current_line:
        split_line = current_line.split(" ")
        data = [int(split_line[i+1].split("=")[1].replace("\n",""),16) for i in range(5)]
        threshold_list.append(current_threshold)
        count_lists.append(data)
        continue
    
    edge_data_lines.append(current_line)
    

count_lists = np.asarray(count_lists)

Counting lines


IOError: [Errno 2] No such file or directory: '20171110-1240_Thresh_scan_start-5_stop-200_step-5_sec_per_rec_120_detector_mode_2E.txt'

In [1]:
# do correlations
corrs_list = []
efficency_percentages = []


for edge_data in edge_data_list:
    # calculate the correlations per section
    corrs = calculate_correlations_from_edge_data(edge_and_tirgger_pos, max_coincidence_time_s, active_channels=active_channels)
    corrs_list.append(corrs)
    # caclate the efficency percentages
    percentages_list = [0 for in range(len(active_channels))]
    for key in corrs.keys():
        if key[0] == '[':
            percentage = 100.0*len(corrs[key]['diff'])/len(corrs[key[1:10]]['diff'])
            percentages_dict[int(key[16])-1] = percentage

efficency_percentages = np.asarray(efficency_percentages)

NameError: name 'edge_data_list' is not defined

In [ ]:
# create folder for the plot dump
directory = "plot_save/"+data_file.split('.')[0]+"/"
if not os.path.exists(directory):
    os.makedirs(directory)

# create plots
for i in range(len(count_lists[0])):
    if count_lists[:,i].any() == 0:
        continue
    if i == 4:
        plt.plot(threshold_list, count_lists[:,i], label="Threefold trigger")
    else:
        plt.plot(threshold_list, count_lists[:,i], label="CH"+str(i))
    plt.legend()
    title = "Threshold scan - CH"+str(i)
    plt.title(title)
    plt.xlabel("Threshold [mV]")
    plt.ylabel("Counts [1]")
    plt.savefig(directory+title+".png")
    plt.show()

for i in range(len(count_lists[0])):
    if count_lists[:,i].any() == 0:
        continue
    if i == 4:
        plt.semilogy(threshold_list, count_lists[:,i], label="Threefold trigger")
    else:
        plt.semilogy(threshold_list, count_lists[:,i], label="CH"+str(i))
plt.legend()
title = "All Channels - logarithmic y-axis"
plt.title(title)
plt.xlabel("Threshold [mV]")
plt.ylabel("Counts [1]")
plt.savefig(directory+title+".png")
plt.show()